In [68]:
import sys
import os
import time
#pour insérer des lignes dans une DB 
from io import StringIO
sys.path.append('/Users/focus_profond/GIT_repo/flight_price_tracker')
#Importing personal modules
from Modules.metadata_functions import *
from Modules.DF_functions import *
import psycopg2
import openpyxl

In [103]:
# ON RECUPERE LA TABLE BRONZE
name_folder = '/Users/focus_profond/GIT_repo/flight_price_tracker/Data/silver/BigTable'
df_raw = DeltaTable(name_folder).to_pandas()
df_raw.info() # 225 394
df_raw['flight_date'].unique()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225394 entries, 0 to 225393
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   flight_date     225394 non-null  object
 1   flight_price    225394 non-null  object
 2   date_of_search  225394 non-null  object
 3   trip            225394 non-null  object
 4   file_name       225394 non-null  object
 5   is_error        225394 non-null  bool  
 6   id              225394 non-null  int64 
 7   currency        225394 non-null  object
 8   country_ori     225394 non-null  object
 9   country_desti   225394 non-null  object
 10  city_ori        225394 non-null  object
 11  city_desti      225394 non-null  object
 12  codeIATA_ori    225394 non-null  object
 13  codeIATA_desti  225394 non-null  object
dtypes: bool(1), int64(1), object(12)
memory usage: 22.6+ MB


array(['2026-05-28', '2025-06-11', '2025-07-10', '2025-11-16',
       '2025-11-10', '2026-05-14', '2025-09-12', '2025-08-08',
       '2025-07-13', '2025-06-16', '2026-04-25', '2026-02-03',
       '2026-05-17', '2025-09-29', '2025-08-15', '2026-05-12',
       '2026-04-14', '2025-09-11', '2025-08-24', '2026-02-25',
       '2025-07-05', '2025-10-23', '2025-11-13', '2025-09-03',
       '2026-02-08', '2025-07-27', '2025-06-20', '2026-04-30',
       '2026-02-19', '2025-07-03', '2025-10-29', '2025-11-08',
       '2026-03-25', '2026-01-08', '2025-07-15', '2025-06-06',
       '2025-07-07', '2025-09-26', '2025-12-06', '2026-01-10',
       '2025-06-27', '2026-03-04', '2026-05-08', '2026-01-21',
       '2025-10-01', '2026-03-29', '2026-05-04', '2026-01-12',
       '2025-10-19', '2025-06-28', '2025-08-30', '2025-12-04',
       '2025-07-21', '2026-05-25', '2026-05-09', '2026-04-27',
       '2025-10-06', '2025-06-09', '2026-03-28', '2025-12-05',
       '2026-01-14', '2025-11-02', '2025-06-18', '2026-

In [ ]:
# ADDING THE ID COLUMN OF THE RAW BIG TABLE
#inserting bigtable into the bronze layer
df_raw = df_raw.sort_index(ascending=False)
df_raw = df_raw.reset_index(drop=True) # pour s'assurer que les index sont consécutifs
df_raw['id']= df_raw.index+1  # Commence à 1 (ou à 0 si tu préfères)
#max(df['id'])
df_raw = df_raw.loc[df_raw['date_of_search']< '2025-06-17']
df_raw.tail(2)
df_raw.info() # 225 394



<class 'pandas.core.frame.DataFrame'>
Index: 224238 entries, 0 to 224237
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   flight_date     224175 non-null  object
 1   flight_price    224238 non-null  object
 2   date_of_search  224238 non-null  object
 3   trip            224238 non-null  object
 4   file_name       224238 non-null  object
 5   is_error        224238 non-null  bool  
 6   id              224238 non-null  int64 
dtypes: bool(1), int64(1), object(5)
memory usage: 12.2+ MB


In [78]:
# ADDING COLUMNS TO THE BIG TABLE
# origin_city
# destination_city
# origin_country
# destination_country

# Ouvre le fichier Excel
# FULL PATH NOT RELATIVE PATH
df_loca = pd.read_excel("/Users/focus_profond/GIT_repo/flight_price_tracker/Config/trip_config.xlsx", sheet_name='good_one')
df_loca.head()
df_raw = pd.merge(df_raw, df_loca, how='left', on='trip')
df_raw = df_raw.drop(columns = ['url'])
df_raw.head()

,flight_date,flight_price,date_of_search,trip,file_name,is_error,id,country_ori,country_desti,city_ori,city_desti,codeIATA_ori,codeIATA_desti
0,2025-06-30,€940,2025-05-26,EZE_BRU,2025-05-26_XxX_EZE_BRU_XxX_06_2025.png,False,1,Argentina,Belgium,Buenos Aires,Brussels,EZE,BRU
1,2025-06-29,€974,2025-05-26,EZE_BRU,2025-05-26_XxX_EZE_BRU_XxX_06_2025.png,False,2,Argentina,Belgium,Buenos Aires,Brussels,EZE,BRU
2,2025-06-28,"€1,070",2025-05-26,EZE_BRU,2025-05-26_XxX_EZE_BRU_XxX_06_2025.png,False,3,Argentina,Belgium,Buenos Aires,Brussels,EZE,BRU
3,2025-06-27,€793,2025-05-26,EZE_BRU,2025-05-26_XxX_EZE_BRU_XxX_06_2025.png,False,4,Argentina,Belgium,Buenos Aires,Brussels,EZE,BRU
4,2025-06-26,€793,2025-05-26,EZE_BRU,2025-05-26_XxX_EZE_BRU_XxX_06_2025.png,False,5,Argentina,Belgium,Buenos Aires,Brussels,EZE,BRU


In [83]:
# UPDATING THE SILVER BIG TABLE DEPENDING ON THE ID
name_folder = '/Users/focus_profond/GIT_repo/flight_price_tracker/Data/silver/BigTable'
partition_cols = None
predicate = "target.id = source.id"
source = 'bronze_layer'
author = 'Augustin'
save_new_data_as_delta(df_raw,name_folder,predicate= predicate, partition_cols=partition_cols, layer = 'Silver', source= source, author =author)

In [93]:
df_test = DeltaTable(name_folder).to_pandas()
df_test['currency'] = '€'
df_test.tail(2)
#df_test.info()  # 224 238  # 225 394

,flight_date,flight_price,date_of_search,trip,file_name,is_error,id,currency
225392,2026-03-27,€465,2025-06-13,EZE_PTY,2025-06-13_XxX_EZE_PTY_XxX_03_2026.png,False,224237,€
225393,2026-03-09,€365,2025-06-13,EZE_PTY,2025-06-13_XxX_EZE_PTY_XxX_03_2026.png,False,224238,€


In [31]:
#FILTERING THE 
#CONFIGURATION DB
#connextion to the DB
DB_HOST = "localhost"
DB_PORT = 5432
DB_NAME = "idle_city"
DB_USER = "postgres"
DB_PASS = "ym_grePO"
DB_SCHEMA = "FPT"  #flight price tracker
my_conn = psycopg2.connect(
    database = DB_NAME
	,user= DB_USER
	,password= DB_PASS
	,host= DB_HOST
	,port= DB_PORT
)
cur = my_conn.cursor()
sql_query = """SELECT max(date_of_search) max_date
                      ,max(id) max_id 
	FROM "FPT".big_table;"""
cur.execute(sql_query)
records = cur.fetchone()
max_date = records[0]
max_id = records[1]
print(max_date, max_id)

2025-06-13 224238


In [16]:
df_to_insert  = df.loc[df['flight_date']> max_date]
df_to_insert.head()

,flight_date,flight_price,date_of_search,trip,file_name,is_error
0,2026-01-04,"€1,400",2025-06-17,EZE_BRU,2025-06-17_XxX_EZE_BRU_XxX_01_2026.png,False
1,2026-01-06,"€1,110",2025-06-17,EZE_BRU,2025-06-17_XxX_EZE_BRU_XxX_01_2026.png,False
2,2026-01-19,€912,2025-06-17,EZE_BRU,2025-06-17_XxX_EZE_BRU_XxX_01_2026.png,False
3,2026-01-21,€912,2025-06-17,EZE_BRU,2025-06-17_XxX_EZE_BRU_XxX_01_2026.png,False
4,2025-07-03,€999,2025-06-17,EZE_BRU,2025-06-17_XxX_EZE_BRU_XxX_07_2025.png,False


In [27]:
#modifying the dataframe

#df['flight_price'] = df['flight_price'].str.replace('€','', regex=False).str.replace(',','')
#on utilise un regex pour que ca soit plus propre
df['flight_price'] = df['flight_price'].str.replace(r"\D","", regex=True)
#df['flight_price'] = df["flight_price"].str.replace('','no_price_founded', regex=False)
df.loc[df['flight_price'].str.strip() == '', 'flight_price'] = 'no_price_founded'
df['flight_date'] = df["flight_date"].fillna("no_date_founded")
nb_of_price_errors = len(df[df["flight_price"] == "no_price_founded"])
nb_of_date_errors = len(df[df["flight_date"] == "no_date_founded"])
df.head(5)


,flight_date,flight_price,date_of_search,trip,file_name,is_error,id
0,2026-01-04,1400,2025-06-17,EZE_BRU,2025-06-17_XxX_EZE_BRU_XxX_01_2026.png,False,1
1,2026-01-06,1110,2025-06-17,EZE_BRU,2025-06-17_XxX_EZE_BRU_XxX_01_2026.png,False,2
2,2026-01-19,912,2025-06-17,EZE_BRU,2025-06-17_XxX_EZE_BRU_XxX_01_2026.png,False,3
3,2026-01-21,912,2025-06-17,EZE_BRU,2025-06-17_XxX_EZE_BRU_XxX_01_2026.png,False,4
4,2025-07-03,999,2025-06-17,EZE_BRU,2025-06-17_XxX_EZE_BRU_XxX_07_2025.png,False,5


In [130]:
df['flight_price'].unique()

array(['', '829', '563', ..., '2210', '110', '95'], dtype=object)

In [26]:
#CONFIGURATION DB
DB_HOST = "localhost"
DB_PORT = 5432
DB_NAME = "idle_city"
DB_USER = "postgres"
DB_PASS = "ym_grePO"
DB_SCHEMA = "FPT"  #flight price tracker
my_conn = psycopg2.connect(
    database = DB_NAME
	,user= DB_USER
	,password= DB_PASS
	,host= DB_HOST
	,port= DB_PORT
)
cur = my_conn.cursor()
drop_query = f'DROP TABLE IF EXISTS "{DB_SCHEMA}".BIG_TABLE CASCADE;'
cur.execute(drop_query)
create_big_table = f"""
    CREATE TABLE "{DB_SCHEMA}".BIG_TABLE
    (
    flight_date TEXT
    ,flight_price TEXT
    ,date_of_search TEXT
    ,trip TEXT
    ,file_name TEXT
    ,is_error BOOLEAN
    ,id SERIAL PRIMARY KEY
    )
"""
cur.execute(create_big_table)
my_conn.commit()
cur.close()
my_conn.close()

In [28]:
#il pourrait être interessant de plutot utiliser des dates que du texte.
my_conn = psycopg2.connect(
    database = DB_NAME
	,user= DB_USER
	,password= DB_PASS
	,host= DB_HOST
	,port= DB_PORT
)
cur = my_conn.cursor()
# 4. Insertion des données avec COPY
buffer = StringIO()
df.to_csv(buffer, index=False, header=False)
buffer.seek(0)
table_name = 'BIG_TABLE'
copy_query = f"""COPY "{DB_SCHEMA}".{table_name} ({', '.join(df.columns)}) FROM STDIN WITH CSV"""
cur.copy_expert(copy_query, buffer)
my_conn.commit()
cur.close()
my_conn.close()


In [47]:
my_conn.commit()
cur.close()
my_conn.close()

In [102]:
my_df= df.copy()

In [ ]:
my_df['flight_price'] = my_df['flight_price'].str.replace('€','', regex=False).str.replace(',','')
my_df.head()

,flight_date,flight_price,date_of_search,trip,file_name,is_error
0,2025-09-01,253,2025-06-12,EZE_SSA,2025-06-12_XxX_EZE_SSA_XxX_09_2025.png,False
1,2025-09-18,253,2025-06-12,EZE_SSA,2025-06-12_XxX_EZE_SSA_XxX_09_2025.png,False
2,2025-09-24,305,2025-06-12,EZE_SSA,2025-06-12_XxX_EZE_SSA_XxX_09_2025.png,False
3,2025-08-03,305,2025-06-12,EZE_SSA,2025-06-12_XxX_EZE_SSA_XxX_08_2025.png,False
4,2025-08-07,225,2025-06-12,EZE_SSA,2025-06-12_XxX_EZE_SSA_XxX_08_2025.png,False


In [104]:
my_df_error = my_df.loc[my_df['is_error']==True]
nb_of_errors = len(my_df_error)
print(nb_of_errors)
my_df_error.sort_values("flight_date").tail()

546


,flight_date,flight_price,date_of_search,trip,file_name,is_error
193454,None,no_price_founded,2025-05-26,AKL_BRU,2025-05-26_XxX_AKL_BRU_XxX_06_2025.png,True
193455,None,18,2025-05-26,AKL_BRU,2025-05-26_XxX_AKL_BRU_XxX_06_2025.png,True
197770,None,712,2025-05-26,BRU_AKL,2025-05-26_XxX_BRU_AKL_XxX_06_2025.png,True
197771,None,no_price_founded,2025-05-26,BRU_AKL,2025-05-26_XxX_BRU_AKL_XxX_06_2025.png,True
197773,None,18,2025-05-26,BRU_AKL,2025-05-26_XxX_BRU_AKL_XxX_06_2025.png,True


In [ ]:
#replace empty value per default value
my_df['flight_date'] = my_df["flight_date"].fillna("no_date_founded")
my_df.head()

,flight_date,flight_price,date_of_search,trip,file_name,is_error
2491,no_date_founded,no_price_founded,2025-06-12,BRU_AKL,2025-06-12_XxX_BRU_AKL_XxX_06_2025.png,True
2492,no_date_founded,no_price_founded,2025-06-12,BRU_AKL,2025-06-12_XxX_BRU_AKL_XxX_06_2025.png,True
2493,no_date_founded,19,2025-06-12,BRU_AKL,2025-06-12_XxX_BRU_AKL_XxX_06_2025.png,True
15799,no_date_founded,no_price_founded,2025-06-11,BRU_AKL,2025-06-11_XxX_BRU_AKL_XxX_06_2025.png,True
15800,no_date_founded,no_price_founded,2025-06-11,BRU_AKL,2025-06-11_XxX_BRU_AKL_XxX_06_2025.png,True


In [91]:
my_df_price_errors = my_df.loc[my_df['flight_price']=='no_price_founded']
nb_of_price_errors = len(my_df_price_errors)
my_df_price_errors.info()
print(nb_of_price_errors)

<class 'pandas.core.frame.DataFrame'>
Index: 17656 entries, 53 to 198498
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   flight_date     17634 non-null  object
 1   flight_price    17656 non-null  object
 2   date_of_search  17656 non-null  object
 3   trip            17656 non-null  object
 4   file_name       17656 non-null  object
 5   is_error        17656 non-null  bool  
dtypes: bool(1), object(5)
memory usage: 844.9+ KB
17656


In [ ]:
my_df_flight_errors = my_df.loc[my_df['flight_date']==None]
nb_of_flight_errors = len(my_df_flight_errors)
my_df_flight_errors.info()
print(nb_of_flight_errors)

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   flight_date     0 non-null      object
 1   flight_price    0 non-null      object
 2   date_of_search  0 non-null      object
 3   trip            0 non-null      object
 4   file_name       0 non-null      object
 5   is_error        0 non-null      bool  
dtypes: bool(1), object(5)
memory usage: 0.0+ bytes
0


In [107]:
#my_df['flight_date'] = my_df["flight_date"].fillna("no_date_founded")

my_df= my_df[my_df['flight_date'].isna()]
my_df.head()


,flight_date,flight_price,date_of_search,trip,file_name,is_error


In [99]:
name_folder = '/Users/focus_profond/GIT_repo/flight_price_tracker/Data/bronze/BigTable'
name_folder = '/Users/focus_profond/GIT_repo/flight_price_tracker/Data/silver/BigTable'
df_silver = DeltaTable(name_folder).to_pandas()
maxid = max(df_silver['id'])
print(maxid)

225394
